# [Movielens full dataset EDA](https://grouplens.org/datasets/movielens/)

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
movies_raw = pd.read_csv('./data/movies_big.csv')
ratings_raw = pd.read_csv('./data/ratings_big.csv')
movies_raw.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## b)

In [16]:
ratings_raw['movieId'].value_counts()[:10]

318     97999
356     97040
296     92406
593     87899
2571    84545
260     81815
480     76451
527     71516
110     68803
1       68469
Name: movieId, dtype: int64

In [20]:
movies_raw.columns

Index(['movieId', 'title', 'genres'], dtype='object')

In [48]:
movieId_with_most_ratings = ratings_raw['movieId'].value_counts()[:10].rename('number_of_ratings')
movieId_with_most_ratings

318     97999
356     97040
296     92406
593     87899
2571    84545
260     81815
480     76451
527     71516
110     68803
1       68469
Name: number_of_ratings, dtype: int64

In [50]:
movies_raw.query(f"movieId == {list(movieId_with_most_ratings.index)}").set_index('movieId').join(movieId_with_most_ratings).sort_values('number_of_ratings', ascending=False)

,title,genres,number_of_ratings
movieId,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,97999
356,Forrest Gump (1994),Comedy|Drama|Romance|War,97040
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,92406
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,87899
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,84545
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,81815
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,76451
527,Schindler's List (1993),Drama|War,71516
110,Braveheart (1995),Action|Drama|War,68803
